In [2]:
!pip install torchsummary

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F



class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        
        self.conv1 = nn.Conv1d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        # self.bn1 = nn.GroupNorm(planes//2, planes)
        self.bn1 = nn.BatchNorm1d(planes)

        self.conv2 = nn.Conv1d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        # self.bn2 = nn.GroupNorm(planes//2, planes)
        self.bn2 = nn.BatchNorm1d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                    nn.Conv1d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                    nn.BatchNorm1d(self.expansion*planes)
                    )


    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out



class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        
        
        self.conv1 = nn.Conv1d(in_planes, planes, kernel_size=1, bias=False)
        # self.bn1 = nn.GroupNorm(planes//2, planes)
        self.bn1 = nn.BatchNorm1d(planes)

        self.conv2 = nn.Conv1d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        # self.bn2 = nn.GroupNorm(planes//2, planes)
        self.bn2 = nn.BatchNorm1d(planes)

        self.conv3 = nn.Conv1d(planes, self.expansion*planes, kernel_size=1, bias=False)
        # self.bn2 = nn.GroupNorm(self.expansion*planes//2, self.expansion*planes)
        self.bn3 = nn.BatchNorm1d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                    nn.Conv1d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                    nn.BatchNorm1d(self.expansion*planes)
                    )


    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out




class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=3):
        super(ResNet, self).__init__()
        self.in_planes = 128
        
        # self.avg1 = nn.AdaptiveAvgPool1d(5000)
        self.conv1 = nn.Conv1d(1, 128, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm1d(128)

        self.layer1 = self._make_layer(block, 128, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 256, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 512, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 1024, num_blocks[3], stride=2)
        # self.linear = nn.Linear(512*block.expansion, num_classes)
        self.linear1 = nn.Linear(9216*block.expansion, 1024)
        self.linear2 = nn.Linear(1024, num_classes)

    
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion

        return nn.Sequential(*layers)
    

    def forward(self, x):
        
        # out = self.avg1(x)
        out = F.relu(self.bn1(self.conv1(x)))
 
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        #out = F.avg_pool1d(out, 4)
        out = F.avg_pool1d(out, 16)
        out = out.view(out.size(0), -1)
        out = self.linear1(out)
        out = self.linear2(out)
        # out = self.linear(out)

        return out






def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])


def ResNet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])


def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])







# These architectures are for Nanopore Translocation Signal Features Prediction
# In our case we predict the number of translocation events inside a window in a trace
def ResNet10_Counter():
    return ResNet(BasicBlock, [1, 1, 1, 1], num_classes=1)

def ResNet18_Counter():
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes=1)

def ResNet34_Counter():
    return ResNet(BasicBlock, [3, 4, 6, 3], num_classes=1)

def ResNet50_Counter():
    return ResNet(Bottleneck, [3, 4, 6, 3], num_classes=1)

def ResNet101_Counter():
    return ResNet(Bottleneck, [3, 4, 23, 3], num_classes=1)

def ResNet152_Counter():
    return ResNet(Bottleneck, [3, 8, 36, 3], num_classes=1)



In [15]:
from numpy import vstack
from torchsummary import summary
import numpy as np
from pandas import read_csv
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, r2_score
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.optim import SGD, Adam, Adagrad
import torch.nn.functional as F
from torch import Tensor
from torch.nn import BCEWithLogitsLoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt
import math
import time
 
# dataset definition
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, path, normalize = False):
        global min_y
        global max_y
        # load the csv file as a dataframe
        df = read_csv(path)
        df = df.transpose()
        # store the inputs and outputs
        self.X = df.values[:100, :-1]
        # print(self.X)
        self.y = df.values[:100, -1]      
        #print("Dataset length: ", self.X.shape[0])
        
        # ensure input data is floats
        self.X = self.X.astype(np.float)
        self.y = self.y.astype(np.float)
        # print(self.y)

        if normalize:
            self.X = self.X.reshape(self.X.shape[1], self.X.shape[0])
            min_X = np.min(self.X,0)  # returns an array of means for each beat
            max_X = np.max(self.X,0)
            self.X = (self.X - min_X)/(max_X-min_X)
            min_y = np.min(self.y)
            max_y = np.max(self.y)
            self.y = (self.y - min_y)/(max_y-min_y)
            
            # #UNCOMMENT IN CASE OF OVERFITTING 1 SAMPLE
            # min_X = np.min(self.X)  
            # max_X = np.max(self.X)
            # self.X = (self.X - min_X)/(max_X-min_X)
            # print(self.X)
            # self.X = self.X.reshape(1, 1, 1200)

            
        # reshape input data
        self.X = self.X.reshape(self.X.shape[1], 1, self.X.shape[0])
        self.y = self.y.reshape(self.y.shape[0],)
        # label encode target and ensure the values are floats
        # self.y = LabelEncoder().fit_transform(self.y)
        self.y = self.y.astype(np.float)
 
    # number of rows in the dataset
    def __len__(self):
        return len(self.X)
 
    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]
 
    # get indexes for train and test rows
    def get_splits(self, n_valid=0.2,n_test=0.2):
        # determine sizes
        test_size = round(n_test * len(self.X))
        valid_size = round(n_valid * len(self.X))
        train_size = len(self.X) - test_size - valid_size
        
        train_set, val_set, test_set = random_split(self, [train_size, valid_size, test_size])
        # calculate the split
        return train_set, val_set, test_set
    
 
 
# prepare the dataset
def prepare_data(path,batch_size):
    # load the dataset
    dataset = CSVDataset(path, normalize = True)
    # calculate split
    train, validation, test = dataset.get_splits()
    
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=batch_size, shuffle=False)
    valid_dl = DataLoader(validation, batch_size=batch_size, shuffle=False)
    test_dl = DataLoader(test, batch_size=batch_size, shuffle=False)
 
    return train_dl, valid_dl, test_dl

 
# train the model
def train_model(train_dl, valid_dl, model, learning_rate, momentum, optimizer, model_name):
    global start

    PATH = "./drive/MyDrive/Tesi/model.pt"
    # define the optimization
    # criterion = BCEWithLogitsLoss()
    # criterion = nn.MSELoss()
    criterion = nn.SmoothL1Loss()
    if optimizer == "adam":
        optimizer = Adam(model.parameters(), lr=learning_rate, eps=momentum)
    elif optimizer == "sgd":
        optimizer = SGD(model.parameters(), lr=learning_rate, momentum=momentum)
    elif optimizer == "adagrad":
        optimizer = Adagrad(model.parameters(), lr=learning_rate)
    
    model = model.float()
    if torch.cuda.is_available():
      model.cuda()
      criterion.cuda()
    loss_history = list()
    valid_loss_history = list()
    # enumerate epochs
    start = time.time()

    # checkpoint = torch.load(PATH)
    # model.load_state_dict(checkpoint['model_state_dict'])
    # optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    # epoch = checkpoint['epoch']
    # loss = checkpoint['loss']

    for epoch in range(200):
        # s = "Epoch " + str(epoch+1) + "/5"
        # print(s, end="\t")
        train_loss = 0.0
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(iter(train_dl)):
            targets = torch.reshape(targets, (targets.shape[0], 1))
            # Transfer Data to GPU if available
            if torch.cuda.is_available():
                # print("GPU found")
                inputs, targets = inputs.cuda(), targets.cuda()
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs.float())
            # print("Prediction vs Target", round(yhat.item(),1), targets.item(),1, end = "\t")
            # calculate loss
            loss = criterion(yhat, targets.float())
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()
            train_loss += loss
            
        loss_history.append(train_loss.item())
        # print("train loss: "+str(train_loss.item()),end="\t")
        
        #VALIDATE NETWORK
        valid_loss = 0.0
        #model.eval()     # Optional when not using Model Specific layer
        for i, (inputs, targets) in enumerate(iter(valid_dl)):
            targets = torch.reshape(targets, (targets.shape[0], 1))
            # # Transfer Data to GPU if available
            if torch.cuda.is_available():
              inputs, targets = inputs.cuda(), targets.cuda()
            # Forward Pass
            yhat = model(inputs.float())
            # Find the Loss
            loss = criterion(yhat,targets.float())
            # Calculate Loss
            valid_loss += loss
            
        valid_loss_history.append(valid_loss.item())
        # print("validation loss: "+str(valid_loss.item()),end="\n")
        
        # torch.save({
        #             'epoch': epoch,
        #             'model_state_dict': model.state_dict(),
        #             'optimizer_state_dict': optimizer.state_dict(),
        #             'train_loss': loss_history,
        #             'valid_loss': valid_loss_history,
        #             }, PATH)
            
    # plt.figure()
    # plt.title("Loss history - "+model_name)
    # plt.xlabel("Epochs")
    # plt.grid()
    # plt.ylabel("SmoothL1Loss")
    # plt.plot(loss_history)
    # #plt.savefig("./ResNet_results/loss_" +model_name+".png")
    # plt.show()
    
    # plt.figure()
    # plt.title("Loss history zoom - "+model_name)
    # plt.xlim([25, 300])
    # plt.xlabel("Epochs")
    # plt.grid()
    # plt.ylabel("SmoothL1Loss")
    # plt.plot(loss_history[25:])
    # #plt.savefig("./ResNet_results/loss_zoom" +model_name+".png")
    # plt.show()
    
    # plt.figure()
    # plt.title("Validation loss history - "+model_name)
    # plt.xlabel("Epochs")
    # plt.grid()
    # plt.ylabel("SmoothL1Loss")
    # plt.plot(valid_loss_history)
    # #plt.savefig("./ResNet_results/val_loss_" +model_name+".png")
    # plt.show()

    return train_loss.item()



# evaluate the model
def evaluate_model(test_dl, model, t, title):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(iter(test_dl)):
        if torch.cuda.is_available():
          inputs, targets = inputs.cuda(), targets.cuda()
        # evaluate the model on the test set
        yhat = model(inputs.float())
        # retrieve numpy array
        yhat, targets = yhat.cpu(), targets.cpu()
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        # round to class values
        #yhat = yhat.round()
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    y_hat = predictions * (max_y-min_y) + min_y
    y = actuals * (max_y-min_y) + min_y
    # calculate accuracy
    acc = math.sqrt(mean_squared_error(actuals, predictions))
    acc_denorm = math.sqrt(mean_squared_error(y, y_hat))
    r2 = r2_score(y, y_hat)
    
    end = time.time()
    runtime = end - start
    
    plt.figure()
    plt.plot(np.linspace(25, 300), np.linspace(25, 300), 'magenta')
    plt.scatter(y,y_hat, s=5, color='indigo')
    plt.xlabel("Real")
    plt.ylabel("Predicted")
    plt.title("Real vs Predicted " + t + " - " + title)
    plt.grid()
    #plt.savefig("./ResNet_results/scatter_" +title+t+".png")
    plt.show()
    
    return acc, acc_denorm, r2, runtime, y-y_hat

 
# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

In [ ]:
import pickle
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

def run_model(hyperparams):

    bs=hyperparams["bs"]
    lr=hyperparams["lr"]
    momentum=hyperparams["momentum"]
    optimizer=hyperparams["optimizer"]
    model_name = hyperparams["model"]
    print(hyperparams)

    path = './drive/MyDrive/Tesi/dataset_ratio.csv'
    if model_name == "ResNet18":
      model = ResNet18_Counter()
    elif model_name == "ResNet34":
      model = ResNet34_Counter()

    train_dl, valid_dl, test_dl = prepare_data(path,batch_size=bs)
    # train the model
    train_loss = train_model(train_dl, valid_dl, model, lr, momentum, optimizer, model_name)
    print(train_loss)
    
    return train_loss

# trials = Trials()

with open('./drive/MyDrive/Tesi/trials.pickle', 'rb') as handle:
    trials = pickle.load(handle)

space = {
    "bs": hp.choice("bs", [4,8,16,32,64,128]),
    "lr": hp.choice("lr", [1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 0.1]),
    "momentum": hp.choice("momentum", [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
    "optimizer": hp.choice("optimizer", ["adam","sgd"]),
    "model": hp.choice("model", ["ResNet18", "ResNet34"])
}

best = fmin(
    fn=run_model,  
    space=space,
    algo=tpe.suggest,
    trials=trials,
    max_evals=24
)

with open('./drive/MyDrive/Tesi/trials.pickle', 'wb') as handle:
    pickle.dump(trials, handle)
print(f"Optimal value of x: {best}")



{'bs': 8, 'lr': 1e-08, 'model': 'ResNet18', 'momentum': 0.2, 'optimizer': 'adam'}
2.632268190383911
{'bs': 16, 'lr': 0.001, 'model': 'ResNet34', 'momentum': 0.2, 'optimizer': 'adam'}
3.3731873827491654e-07
{'bs': 8, 'lr': 1e-10, 'model': 'ResNet18', 'momentum': 0.8, 'optimizer': 'adam'}
1.6605051755905151
{'bs': 4, 'lr': 1e-07, 'model': 'ResNet18', 'momentum': 0.7, 'optimizer': 'adam'}
0.7123553156852722
{'bs': 16, 'lr': 1e-08, 'model': 'ResNet34', 'momentum': 0.3, 'optimizer': 'sgd'}
0.3848758637905121
{'bs': 128, 'lr': 0.1, 'model': 'ResNet18', 'momentum': 0.7, 'optimizer': 'sgd'}
nan
{'bs': 64, 'lr': 1e-06, 'model': 'ResNet18', 'momentum': 0.5, 'optimizer': 'sgd'}
0.1043463796377182
{'bs': 128, 'lr': 1e-09, 'model': 'ResNet18', 'momentum': 0.8, 'optimizer': 'adam'}
 35%|███▌      | 7/20 [45:12<1:13:54, 341.09s/it, best loss: 3.3731873827491654e-07]

In [ ]:
from torchsummary import summary
import matplotlib.pyplot as plt

model = ResNet18_Counter()
path = './drive/MyDrive/Tesi/dataset_ratio.csv'
model_name = "ResNet18"

train_dl, valid_dl, test_dl = prepare_data(path,4)
print("Dataset Prepared")
# train the model
train_model(train_dl, valid_dl, model, 1e-3, "adam", model_name)
# evaluate the model on training
acc_train, acc_denorm_train, r2_train, runtime, error_train = evaluate_model(train_dl, model, "train", model_name)
print("")
print('Train RMSE: %.3f' % acc_train)
print('Train RMSE (denorm): %.3f' % acc_denorm_train)
print('Train R2 score: %.3f' %r2_train)

# evaluate the model on validation
acc_valid, acc_denorm_valid, r2_valid, runtime, error_valid = evaluate_model(valid_dl, model, "validation", model_name)
print("")
print('Validation RMSE: %.3f' % acc_valid)
print('Validation RMSE (denorm): %.3f' % acc_denorm_valid)
print('Validation R2 score: %.3f' %r2_valid)

# evaluate model on test
acc_test, acc_denorm_test, r2_test, runtime_test, error_test = evaluate_model(test_dl, model, "test", model_name)
print('Test RMSE: %.3f' % acc_test)
print('Test RMSE (denorm): %.3f' % acc_denorm_test)
print('Test R2 score: %.3f' %r2_test)
print("Execution time: %.3f" %runtime)

plt.figure()
plt.hist(error_train, 50, color='dodgerblue')
plt.hist(error_test, 50, color='gold')
plt.xlabel('Error on estimation')
plt.ylabel('Frequency')
plt.legend(["Train","Test"])
plt.title('Histogram for the estimation error - ' + model_name)
#plt.savefig("./ResNet_results/hist_" +model_name+".png")
plt.show()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')